In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re

from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore') 
%matplotlib inline

from functions.custom_function_5 import remove_pattern
from functions.custom_function_5 import counting
from functions.custom_function_5 import nltk_tag_to_wordnet_tag
from functions.custom_function_5 import lemmatize_sentence

#  Merging dataframes of positive tweets with negative

In [2]:
# importing initial dataset
df_1 = pd.read_csv('../data/raw/labeled_data.csv')

The following dataset had 6 columns:

1) count = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

2) hate_speech = number of CF users who judged the tweet to be hate speech.

3) offensive_language = number of CF users who judged the tweet to be offensive.

4)neither = number of CF users who judged the tweet to be neither offensive nor non-offensive.

5)class = class label for majority of CF users. 0 - **hate speech** / 1 - **offensive language** / 2 - **neither**

6)tweet = content of the tweet

In [3]:
# printing out the info
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
Unnamed: 0            24783 non-null int64
count                 24783 non-null int64
hate_speech           24783 non-null int64
offensive_language    24783 non-null int64
neither               24783 non-null int64
class                 24783 non-null int64
tweet                 24783 non-null object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [4]:
# renaming the first columns as id
df_1.rename(columns={'Unnamed: 0':'id'}, inplace = True)
# increasing max length for all columns and number of columns
pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_columns", 50)

pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_rows', 500)

In [5]:
# set id to index
df_1.set_index('id').head(5)

,count,hate_speech,offensive_language,neither,class,tweet
id,,,,,,
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;


In [6]:
# there are three different classes on this dataset, all of which vaguely or clearly offensive
df_1['class'].value_counts()

1    19190
2    4163 
0    1430 
Name: class, dtype: int64

In the next section I'll take a closer look at some of the comments for each class in order to see whether the count/vote system is represantitive of the actual Tweet content

In [7]:
# racist or homophobic comments
hate_speech = df_1.loc[df_1['class']==0] 

In [8]:
hate_speech.head(50)


,id,count,hate_speech,offensive_language,neither,class,tweet
85,85,3,2,1,0,0,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy11 queer"" gaywad"
89,90,3,3,0,0,0,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a beaner smh you can tell hes a mexican"
110,111,3,3,0,0,0,"""@DevilGrimz: @VigxRArts you're fucking gay, blacklisted hoe"" Holding out for #TehGodClan anyway http://t.co/xUCcwoetmn"
184,186,3,3,0,0,0,"""@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPLE https://t.co/RNvD2nLCDR"" This is why there's black people and niggers"
202,204,3,2,1,0,0,"""@NoChillPaz: ""At least I'm not a nigger"" http://t.co/RGJa7CfoiT""\n\nLmfao"
204,206,3,2,1,0,0,"""@NotoriousBM95: @_WhitePonyJr_ Ariza is a snake and a coward"" but at least he isn't a cripple like your hero Roach lmaoo"
219,221,6,3,2,1,0,"""@RTNBA: Drakes new shoes that will be released by Nike/Jordan.... Yes, there's glitter on the shoes http://t.co/QCtPLxHEXM"" ....dudes a fag"
260,263,3,2,1,0,0,"""@TheoMaxximus: #GerrysHalloweenParty http://t.co/3ycrSrnjHc"" Halloween was yesterday stupid nigger"
312,317,3,2,1,0,0,"""@ashlingwilde: @ItsNotAdam is bored supposed to be cute, you faggot?"" Sometimes"
315,320,3,2,1,0,0,"""@bigbootybishopp: @white_thunduh lassen cc , the coach wants me"" i feel it youll get more my nig no doubt"


In [9]:
# neither offensive nor non-offensive speech
neutral = df_1.loc[df_1['class']== 2] 

In [10]:
neutral.head(50)

,id,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
40,40,3,0,1,2,2,""" momma said no pussy cats inside my doghouse """
63,63,3,0,0,3,2,"""@Addicted2Guys: -SimplyAddictedToGuys http://t.co/1jL4hi8ZMF"" woof woof hot scally lad"
66,66,3,0,1,2,2,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woof woof and hot soles"
67,67,3,0,1,2,2,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these dishes."" One oreo? Lol"
70,70,3,0,0,3,2,"""@ArizonasFinest6: Why the eggplant emoji doe?""y he say she looked like scream lmao"
75,75,3,0,1,2,2,"""@BabyAnimalPics: baby monkey bathtime http://t.co/7KPWAdLF0R""\nAwwwwe! This is soooo ADORABLE!"
115,116,3,0,0,3,2,"""@DomWorldPeace: Baseball season for the win. #Yankees"" This is where the love started"
118,119,3,0,0,3,2,"""@DunderbaIl: I'm an early bird and I'm a night owl, so I'm wise and have worms."""
119,120,3,0,1,2,2,"""@EdgarPixar: Overdosing on heavy drugs doesn't sound bad tonight."" I do that pussy shit every day."


In [11]:
neutral['count'].value_counts()

3    3919
6    184 
4    37  
9    19  
7    4   
Name: count, dtype: int64

In [12]:
# offensive but not hate speech
offensive_language = df_1.loc[df_1['class']==1] 

In [13]:
offensive_language.head(50)

,id,count,hate_speech,offensive_language,neither,class,tweet
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"""
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up and HATE on another bitch .. I got too much shit going on!"""
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of you big bitches coming for us skinny girls!!&#8221;
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; thats that """
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Mariam""\n\nbitch"
10,10,3,0,3,0,1,""" Keeks is a bitch she curves everyone "" lol I walked into a conversation like this. Smh"


In [14]:
# inserting a new column in our dataframe to account for positive tweets (which we will extract in teh following section)
df_1.insert(5, 'positive' ,0)


In [15]:
df_1['positive'].value_counts()


0    24783
Name: positive, dtype: int64

In [16]:
df_1.tail(5)

,id,count,hate_speech,offensive_language,neither,positive,class,tweet
24778,25291,3,0,2,1,0,1,"you's a muthaf***in lie &#8220;@LifeAsKing: @20_Pearls @corey_emanuel right! His TL is trash &#8230;. Now, mine? Bible scriptures and hymns&#8221;"
24779,25292,3,0,1,2,0,2,"you've gone and broke the wrong heart baby, and drove me redneck crazy"
24780,25294,3,0,3,0,0,1,young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again
24781,25295,6,0,6,0,0,1,youu got wild bitches tellin you lies
24782,25296,3,0,0,3,0,2,"~~Ruffled | Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow &amp; white. A Coll http://t.co/H0dYEBvnZB"


##   Introducing the second Dataset

At this point, we notice that our datapoints only contain either offensive language (of some form) or at least not non-offensive language labelled as neither. In order to build a model that can also successfully identify completely not offensive speech, we'll need to introduce some example of not-offensive language.

The following dataset has only 3 columns:

id: unique ID for each tweet

label: offensive or not

tweet: content of the tweet

In [17]:
# importing a second dataset with only two types of tweets (offensive -1 / not offensive-0)
df_2 = pd.read_csv('../data/raw/train_E6oV3lV.csv')

In [18]:
# only three columns 
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
id       31962 non-null int64
label    31962 non-null int64
tweet    31962 non-null object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [19]:
# the majority of our datapoints are not offensive
df_2['label'].value_counts()

0    29720
1    2242 
Name: label, dtype: int64

In [20]:
# given that we have 23,353 offensive tweets on the other dataset, we'll select 23,000 thousand positive tweets from df_2
df_pos = df_2.loc[df_2['label'] == 0]

In [21]:
df_pos.head(50)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo
6,7,0,@user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
7,8,0,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl
8,9,0,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦
9,10,0,@user @user welcome here ! i'm it's so #gr8 !


In [22]:
# selecting only 23000 to have a balanced dataset
df_p_23 = df_pos[:23000]

In [23]:
len(df_p_23)

23000

In [24]:
# setting the index as id
df_p_23.set_index('id')


,label,tweet
id,,
1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
3,0,bihday your majesty
4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
5,0,factsguide: society now #motivation
...,...,...
24725,0,"yes, leeds!! #lesbiunion #girlsweekend"
24726,0,in other non tattoo related news my boy will be #crystalpalace mascot at the home game against liverpool next season ! ð´ðµ
24727,0,finally wo agaya :):)


In [25]:
# recreating the same structure of the other df
df_p_23.insert(1, 'count' ,3)
df_p_23.insert(2,'hate_speech',0)
df_p_23.insert(3,'offensive_language',0)
df_p_23.insert(4,'neither',0)
df_p_23.insert(5,'positive',3)

In [26]:
df_p_23

,id,count,hate_speech,offensive_language,neither,positive,label,tweet
0,1,3,0,0,0,3,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,3,0,0,0,3,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,3,0,0,0,3,0,bihday your majesty
3,4,3,0,0,0,3,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,3,0,0,0,3,0,factsguide: society now #motivation
...,...,...,...,...,...,...,...,...
24724,24725,3,0,0,0,3,0,"yes, leeds!! #lesbiunion #girlsweekend"
24725,24726,3,0,0,0,3,0,in other non tattoo related news my boy will be #crystalpalace mascot at the home game against liverpool next season ! ð´ðµ
24726,24727,3,0,0,0,3,0,finally wo agaya :):)
24727,24728,3,0,0,0,3,0,ðð ð #love #instagood #photooftheday top.tags #tbt #cute #me #beautiful #followme #followâ¦


In [27]:
# removing the existing label
df_p_23.drop(['label'], axis = 1, inplace = True)

In [28]:
# inserting label as 4 for positive tweets
df_p_23.insert(6,'class', 3)


In [29]:
df_p_23.head(10)

,id,count,hate_speech,offensive_language,neither,positive,class,tweet
0,1,3,0,0,0,3,3,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,3,0,0,0,3,3,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,3,0,0,0,3,3,bihday your majesty
3,4,3,0,0,0,3,3,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,3,0,0,0,3,3,factsguide: society now #motivation
5,6,3,0,0,0,3,3,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo
6,7,3,0,0,0,3,3,@user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
7,8,3,0,0,0,3,3,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl
8,9,3,0,0,0,3,3,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦
9,10,3,0,0,0,3,3,@user @user welcome here ! i'm it's so #gr8 !


In [30]:
# making sure all Ids are unique. Last Id from df_1 was 25926
df_p_23['id'] = df_p_23['id'] + 25926

In [31]:
# final df comprised of 23000 positive tweets
df_p_23.head(10)

,id,count,hate_speech,offensive_language,neither,positive,class,tweet
0,25927,3,0,0,0,3,3,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,25928,3,0,0,0,3,3,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,25929,3,0,0,0,3,3,bihday your majesty
3,25930,3,0,0,0,3,3,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,25931,3,0,0,0,3,3,factsguide: society now #motivation
5,25932,3,0,0,0,3,3,[2/2] huge fan fare and big talking before they leave. chaos and pay disputes when they get there. #allshowandnogo
6,25933,3,0,0,0,3,3,@user camping tomorrow @user @user @user @user @user @user @user dannyâ¦
7,25934,3,0,0,0,3,3,the next school year is the year for exams.ð¯ can't think about that ð­ #school #exams #hate #imagine #actorslife #revolutionschool #girl
8,25935,3,0,0,0,3,3,we won!!! love the land!!! #allin #cavs #champions #cleveland #clevelandcavaliers â¦
9,25936,3,0,0,0,3,3,@user @user welcome here ! i'm it's so #gr8 !


We'll then merge the negative plus positives tweets to have our initial unstructured dataset including both negative and positives tweets

In [32]:
df = pd.concat([df_1, df_p_23], join = 'inner')

In [33]:
df.set_index('id', inplace = True)

In [34]:
# the final dataset has a good balance between positive and negatives tweets
df['positive'].value_counts()

0    24783
3    23000
Name: positive, dtype: int64

In [35]:
df['positive'].value_counts()

0    24783
3    23000
Name: positive, dtype: int64

In [36]:
# reshuffling the rows
df = df.sample(frac=1, random_state=43).reset_index(drop=True)

In [37]:
# final df to be cleaned
len(df)

47783

In [38]:
df['positive'].value_counts()

0    24783
3    23000
Name: positive, dtype: int64

# Data Cleaning

As first step we need to make all tweets lower case so that our model doesn't consider the same words with capital and lower initials as different.

In [39]:
df['tweet_low'] = df['tweet'].apply(lambda x: x if type(x)!=str else x.lower())
df

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.
...,...,...,...,...,...,...,...,...
47778,3,0,0,0,3,3,when quay collab with @user says sold out!!!ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð© #noooo #whyyyy #loveyoudesi #,when quay collab with @user says sold out!!!ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð© #noooo #whyyyy #loveyoudesi #
47779,3,0,3,0,0,1,RT @_groovymovie: &#8220;@Shane_A1: Hmu talmbout match but when I pull up its 30 other niccas wit ya &#128530;&#8221; smfh shit like that kill me,rt @_groovymovie: &#8220;@shane_a1: hmu talmbout match but when i pull up its 30 other niccas wit ya &#128530;&#8221; smfh shit like that kill me
47780,3,0,0,0,3,3,angry squeaking frog video: #frog #nature #animals #cute #adorable,angry squeaking frog video: #frog #nature #animals #cute #adorable
47781,3,0,3,0,0,1,RT @obey_jrock__: This is a true ride or die bitch &#128175; http://t.co/y1t8CTQn4U,rt @obey_jrock__: this is a true ride or die bitch &#128175; http://t.co/y1t8ctqn4u


In [40]:
len(df)

47783

# Checking for duplicates

In [41]:
duplicates = df[df.duplicated('tweet_low', keep = 'first')]
duplicates

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low
303,3,0,0,0,3,3,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
544,3,0,0,0,3,3,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
645,3,0,0,0,3,3,i finally found a way how to delete old tweets! you might find it useful as well: #deletetweets,i finally found a way how to delete old tweets! you might find it useful as well: #deletetweets
1027,3,0,0,0,3,3,i finally found a way how to delete old tweets! you might find it useful as well: #deletetweets,i finally found a way how to delete old tweets! you might find it useful as well: #deletetweets
1085,3,0,0,0,3,3,can #lighttherapy help with or #depression? #altwaystoheal #healthy is #happy !!,can #lighttherapy help with or #depression? #altwaystoheal #healthy is #happy !!
...,...,...,...,...,...,...,...,...
47718,3,0,0,0,3,3,@user bihday rg,@user bihday rg
47728,3,0,0,0,3,3,get #up get get #enjoy #music #today #free #apps #free #music,get #up get get #enjoy #music #today #free #apps #free #music
47758,3,0,0,0,3,3,@user the present moment is filled with joy and happiness @user #thinkbigsundaywithmarsha,@user the present moment is filled with joy and happiness @user #thinkbigsundaywithmarsha
47772,3,0,0,0,3,3,ð #love #instagood #photooftheday top.tags #tbt #cute #me #beautiful #followme #followâ¦,ð #love #instagood #photooftheday top.tags #tbt #cute #me #beautiful #followme #followâ¦


In [42]:
# number of duplicates 
len(duplicates)

1589

In [43]:
# Dropping all duplicates
df = df.drop_duplicates(subset = 'tweet_low', keep = 'first')


In [44]:
# checking for correct length. It should be 47783 - 1589 = 46194
len(df)

46194

In [45]:
duplicates = df[df.duplicated('tweet_low', keep = 'first')]
duplicates

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low


In [46]:
# removed duplicates
single_model = df[df['tweet_low'].str.contains(r'model')]
single_model

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low
49,3,0,0,0,3,3,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
368,3,0,0,0,3,3,@user dance like no one is watching â¨ #modelling #fitnesslife #fit #strong #healthy #fitnessâ¦,@user dance like no one is watching â¨ #modelling #fitnesslife #fit #strong #healthy #fitnessâ¦
520,3,0,0,0,3,3,#fathersday 2 all the positive male role model's celebrating today. gift our #spice #rubs &amp; #bbq #sauces @,#fathersday 2 all the positive male role model's celebrating today. gift our #spice #rubs &amp; #bbq #sauces @
575,3,0,0,0,3,3,@user #model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦,@user #model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
1125,3,0,0,0,3,3,@user @user fully agree! expect failure that's what i tell my ss. need to model that!,@user @user fully agree! expect failure that's what i tell my ss. need to model that!
1228,3,0,0,0,3,3,@user dope tune #kilamiti by iam-bars #love #dj #spin #music #media #me #model #tbtâ¦,@user dope tune #kilamiti by iam-bars #love #dj #spin #music #media #me #model #tbtâ¦
2235,3,0,0,0,3,3,finally made it to new blades! lets hope all the models are ok! #modelmaking #aub #nervous #newbladesshow,finally made it to new blades! lets hope all the models are ok! #modelmaking #aub #nervous #newbladesshow
2253,3,0,0,0,3,3,"darling, just fucking own it. âð»ï¸ð¥ððâ¨ðð #photoshoot #photography #model #beautiful #ownitâ¦","darling, just fucking own it. âð»ï¸ð¥ððâ¨ðð #photoshoot #photography #model #beautiful #ownitâ¦"
3231,3,0,3,0,0,1,"Bad bitches from Oakland, she could beeee a model","bad bitches from oakland, she could beeee a model"
3404,3,0,0,0,3,3,çç¬ã #fashion #smiles #love #girl#instagood#follome #xoxo#japan #tokyo #model#jamp!,çç¬ã #fashion #smiles #love #girl#instagood#follome #xoxo#japan #tokyo #model#jamp!


##  Removing re-tweets

Hypelinks in this case represent retweets on each tweet.

In [47]:
retweet =  df[df['tweet_low'].str.contains(r'http://t(?!$)')]
retweet

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low
11,3,1,2,0,0,1,"@HopOnTheBeast i found her right here, good job unfollowing me btw faggot http://t.co/kD3iSeIoLd","@hoponthebeast i found her right here, good job unfollowing me btw faggot http://t.co/kd3iseiold"
45,3,0,3,0,0,1,"RT @TooSexist: Women complain that chivalry is dead. Yes congratulations bitches, you killed it with feminism http://t.co/Mrc82ZUhOn","rt @toosexist: women complain that chivalry is dead. yes congratulations bitches, you killed it with feminism http://t.co/mrc82zuhon"
56,3,0,0,3,0,2,Yankees great Joe DiMaggio reportedly used to beat wife Marilyn Monroe. Here she is in 1954 announcing divorce http://t.co/blS7dalMiF,yankees great joe dimaggio reportedly used to beat wife marilyn monroe. here she is in 1954 announcing divorce http://t.co/bls7dalmif
76,3,0,3,0,0,1,""" pussy is a powerful drug "" &#128517; #HappyHumpDay http://t.co/R8jsymiB5b",""" pussy is a powerful drug "" &#128517; #happyhumpday http://t.co/r8jsymib5b"
83,3,0,0,3,0,2,&#8220;@CaptainYankee2: Two of the best Yankees Derek Jeter and Joe Torre #JoeTorreDay http://t.co/XMyxfDBKOX&#8221; @jordan_luree,&#8220;@captainyankee2: two of the best yankees derek jeter and joe torre #joetorreday http://t.co/xmyxfdbkox&#8221; @jordan_luree
...,...,...,...,...,...,...,...,...
47743,3,0,3,0,0,1,RT @MyDickNeedsCPR: What lonely hoe made this? http://t.co/eEFThf0tvb,rt @mydickneedscpr: what lonely hoe made this? http://t.co/eefthf0tvb
47750,3,0,3,0,0,1,Bored then a hoe! Listening to these fuck ass adults lecturing us with @__vercetti http://t.co/NyBO16RMsh,bored then a hoe! listening to these fuck ass adults lecturing us with @__vercetti http://t.co/nybo16rmsh
47756,4,1,3,0,0,1,trash both ways lol RT @AgdaCoroner: Bitch Killed Herself....Look Like Bill Maher With Makeup on http://t.co/IWLAG2J5Sl,trash both ways lol rt @agdacoroner: bitch killed herself....look like bill maher with makeup on http://t.co/iwlag2j5sl
47781,3,0,3,0,0,1,RT @obey_jrock__: This is a true ride or die bitch &#128175; http://t.co/y1t8CTQn4U,rt @obey_jrock__: this is a true ride or die bitch &#128175; http://t.co/y1t8ctqn4u


Only negative comments have retweets so I can't use retweets as a variable for our model

In [48]:
retweet['class'].value_counts()

1    1754
2    883 
0    112 
Name: class, dtype: int64

Emojis come in form of text in the structure of an hashtag followed by only numeric digits (e.g #128532)

In [49]:
emoji =  df[df['tweet'].str.contains(r'#[0-9]')]
emoji['class'].value_counts()

1    4852
2    994 
3    406 
0    213 
Name: class, dtype: int64

Within the positive tweets ashtag followed by numbers actually have linguistic meanings or represent dates/year

In [50]:
positve_with_emoji = emoji.loc[emoji['class'] == 3]
positve_with_emoji.head(10)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low
51,3,0,0,0,3,3,@user just done my first #2minutebeachclean in ages. also got one wet foot putting a starfish back in the water â­ï¸ð #southwold ð #dâ¦,@user just done my first #2minutebeachclean in ages. also got one wet foot putting a starfish back in the water â­ï¸ð #southwold ð #dâ¦
192,3,0,0,0,3,3,checked in #holiday #croatia #13daystogo,checked in #holiday #croatia #13daystogo
430,3,0,0,0,3,3,"""no one is better than dad."" fathersday!!! #fathersday2016 #fatherslove #disney #disneyfana #101dalmatians","""no one is better than dad."" fathersday!!! #fathersday2016 #fatherslove #disney #disneyfana #101dalmatians"
538,3,0,0,0,3,3,welcome to candjdays! :) #first #video #vlog #checkitout #youtube #couple #9videos #phoenix #az #florida,welcome to candjdays! :) #first #video #vlog #checkitout #youtube #couple #9videos #phoenix #az #florida
594,3,0,0,0,3,3,@user d-7 opening soon #miami #restaurant #saltandsugarcafe #20flaglerstreet,@user d-7 opening soon #miami #restaurant #saltandsugarcafe #20flaglerstreet
610,3,0,0,0,3,3,@user pheonix u10s new look team to play noh walkden tomorrow #1stgame,@user pheonix u10s new look team to play noh walkden tomorrow #1stgame
613,3,0,0,0,3,3,i get to see my daddy today!! #80days #gettingfed,i get to see my daddy today!! #80days #gettingfed
954,3,0,0,0,3,3,popsy &amp; little all ready for @user #10minutestogo,popsy &amp; little all ready for @user #10minutestogo
1200,3,0,0,0,3,3,i'm off to #florida #usa in #july for #3 #weeks woop woop #holiday,i'm off to #florida #usa in #july for #3 #weeks woop woop #holiday
1226,3,0,0,0,3,3,the new baby is on her way! xx #700d #cannon,the new baby is on her way! xx #700d #cannon


Once again the dataset of positive tweets was already stripped of emojis and we cannot include them as a variable towards our final model.

In [51]:
# pattern to delete URLS for RETWEETS
df['no_url'] = np.vectorize(remove_pattern)(df['tweet_low'], "https?://[A-Za-z0-9./]*")

In [52]:
# checking whether URLS have been removed
df.head(15)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.
5,3,0,0,0,3,3,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user
6,3,0,0,0,3,3,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce
7,3,0,0,0,3,3,@user my final legislative session day has officially begun! @user @user #albany,@user my final legislative session day has officially begun! @user @user #albany,@user my final legislative session day has officially begun! @user @user #albany
8,3,2,1,0,0,0,@lucas_wright955 @MichaelGT03 faggots,@lucas_wright955 @michaelgt03 faggots,@lucas_wright955 @michaelgt03 faggots
9,3,0,0,0,3,3,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦


## Removing Twitter handles and count them

In [54]:
df['handle_count'] = np.vectorize(counting)(df['tweet_low'], "@[\w]*")

In [55]:
# the majority of tweets had no handles
df['handle_count'].value_counts()

0     23968
1     15967
2     4176 
3     1344 
4     416  
5     158  
6     89   
8     31   
7     26   
9     12   
10    6    
11    1    
Name: handle_count, dtype: int64

In [56]:
df.head(5)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere,0
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag,1
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,1


In [58]:
# remove twitter handles (@user)
df['no_handle'] = np.vectorize(remove_pattern)(df['no_url'], "@[\w]*")

In [59]:
df.tail(10)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle
47771,3,0,0,0,3,3,@user '' x'mas &amp; my bihday disney ! '' #love #thanks #karen #â¡,@user '' x'mas &amp; my bihday disney ! '' #love #thanks #karen #â¡,@user '' x'mas &amp; my bihday disney ! '' #love #thanks #karen #â¡,1,'' x'mas &amp; my bihday disney ! '' #love #thanks #karen #â¡
47773,3,0,0,0,3,3,my soul is happiest on the water! #soul #happier #happiest #water #ocean #beach #caliâ¦,my soul is happiest on the water! #soul #happier #happiest #water #ocean #beach #caliâ¦,my soul is happiest on the water! #soul #happier #happiest #water #ocean #beach #caliâ¦,0,my soul is happiest on the water! #soul #happier #happiest #water #ocean #beach #caliâ¦
47774,3,0,0,0,3,3,"â #nzd/usd post-rbnz rally almost reversed, 0.7000 closer #blog #silver #gold #forex","â #nzd/usd post-rbnz rally almost reversed, 0.7000 closer #blog #silver #gold #forex","â #nzd/usd post-rbnz rally almost reversed, 0.7000 closer #blog #silver #gold #forex",0,"â #nzd/usd post-rbnz rally almost reversed, 0.7000 closer #blog #silver #gold #forex"
47775,3,0,3,0,0,1,"RT @MAKEUP_SEX: trash talked by many . hated by some . & guess how many fucks i give , its less than one .","rt @makeup_sex: trash talked by many . hated by some . & guess how many fucks i give , its less than one .","rt @makeup_sex: trash talked by many . hated by some . & guess how many fucks i give , its less than one .",1,"rt : trash talked by many . hated by some . & guess how many fucks i give , its less than one ."
47776,3,0,0,0,3,3,"#bihday to leo's mom, #celia ..","#bihday to leo's mom, #celia ..","#bihday to leo's mom, #celia ..",0,"#bihday to leo's mom, #celia .."
47777,3,0,3,0,0,1,@_ElenaRaquel_ its swag bitch aha,@_elenaraquel_ its swag bitch aha,@_elenaraquel_ its swag bitch aha,1,its swag bitch aha
47778,3,0,0,0,3,3,when quay collab with @user says sold out!!!ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð© #noooo #whyyyy #loveyoudesi #,when quay collab with @user says sold out!!!ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð© #noooo #whyyyy #loveyoudesi #,when quay collab with @user says sold out!!!ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð© #noooo #whyyyy #loveyoudesi #,1,when quay collab with says sold out!!!ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð©ð«ð© #noooo #whyyyy #loveyoudesi #
47779,3,0,3,0,0,1,RT @_groovymovie: &#8220;@Shane_A1: Hmu talmbout match but when I pull up its 30 other niccas wit ya &#128530;&#8221; smfh shit like that kill me,rt @_groovymovie: &#8220;@shane_a1: hmu talmbout match but when i pull up its 30 other niccas wit ya &#128530;&#8221; smfh shit like that kill me,rt @_groovymovie: &#8220;@shane_a1: hmu talmbout match but when i pull up its 30 other niccas wit ya &#128530;&#8221; smfh shit like that kill me,2,rt : &#8220;: hmu talmbout match but when i pull up its 30 other niccas wit ya &#128530;&#8221; smfh shit like that kill me
47781,3,0,3,0,0,1,RT @obey_jrock__: This is a true ride or die bitch &#128175; http://t.co/y1t8CTQn4U,rt @obey_jrock__: this is a true ride or die bitch &#128175; http://t.co/y1t8ctqn4u,rt @obey_jrock__: this is a true ride or die bitch &#128175;,1,rt : this is a true ride or die bitch &#128175;
47782,3,0,3,0,0,1,RT @AllHailTaron_: I got the deals for the low. I know you hoes lonely so fuck with these cuffing season specials. &#128184;&#128175; http://t.co/YURpX99Hdb,rt @allhailtaron_: i got the deals for the low. i know you hoes lonely so fuck with these cuffing season specials. &#128184;&#128175; http://t.co/yurpx99hdb,rt @allhailtaron_: i got the deals for the low. i know you hoes lonely so fuck with these cuffing season specials. &#128184;&#128175;,1,rt : i got the deals for the low. i know you hoes lonely so fuck with these cuffing season specials. &#128184;&#128175;


## Removing special characters apart from hashtags and counting tweet lenght

Also, deleting all special characters apart from hastags followed by words since emojis are only applicable to bad tweets.

In [60]:
# getting rid of all characters apart from letters ashtags and apostrophes
df['no_handle_no_special'] = df['no_handle'].str.replace("[^a-zA-Z#']", " ")

In [61]:
df.head(15)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere,0,13 days to go #gettingthere,days to go #gettingthere
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag,1,get off my twitter fag,get off my twitter fag
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery # # #
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the racists cuz she's not white.,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,1,dubya hates todays teabagger movement.,dubya hates todays teabagger movement
5,3,0,0,0,3,3,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,1,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness
6,3,0,0,0,3,3,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce,0,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows #country #music #lylepierce
7,3,0,0,0,3,3,@user my final legislative session day has officially begun! @user @user #albany,@user my final legislative session day has officially begun! @user @user #albany,@user my final legislative session day has officially begun! @user @user #albany,3,my final legislative session day has officially begun! #albany,my final legislative session day has officially begun #albany
8,3,2,1,0,0,0,@lucas_wright955 @MichaelGT03 faggots,@lucas_wright955 @michaelgt03 faggots,@lucas_wright955 @michaelgt03 faggots,2,faggots,faggots
9,3,0,0,0,3,3,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦,0,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦,a #bikini kind of life summer #palmtrees #breeze #place #cali #california #swimwear


In [62]:
# remove single ashtags with nothing following them
df['no_handle_no_special_no_sin_ash'] = np.vectorize(remove_pattern)(df['no_handle_no_special'], " # ")

In [63]:
# counting the length of tweets after URLs are removed, we will use this to see if there's a correlation between length of tweets and sentiment
df['tweet_length'] = df['no_handle_no_special_no_sin_ash'].apply(lambda x: len(x))
df.head(50)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere,0,13 days to go #gettingthere,days to go #gettingthere,days to go #gettingthere,30
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag,1,get off my twitter fag,get off my twitter fag,get off my twitter fag,23
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery # # #,these hoes got more bodies than a cemetery,58
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the racists cuz she's not white.,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,124
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,1,dubya hates todays teabagger movement.,dubya hates todays teabagger movement,dubya hates todays teabagger movement,39
5,3,0,0,0,3,3,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,1,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,117
6,3,0,0,0,3,3,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce,0,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows #country #music #lylepierce,getting for this weekends shows #country #music #lylepierce,62
7,3,0,0,0,3,3,@user my final legislative session day has officially begun! @user @user #albany,@user my final legislative session day has officially begun! @user @user #albany,@user my final legislative session day has officially begun! @user @user #albany,3,my final legislative session day has officially begun! #albany,my final legislative session day has officially begun #albany,my final legislative session day has officially begun #albany,69
8,3,2,1,0,0,0,@lucas_wright955 @MichaelGT03 faggots,@lucas_wright955 @michaelgt03 faggots,@lucas_wright955 @michaelgt03 faggots,2,faggots,faggots,faggots,9
9,3,0,0,0,3,3,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦,a #bikini kind of life ð´ summer #palmtrees #breeze #place #cali #california #swimwearâ¦,0,a #bikini kind of life ð´ sum

Checking there are no outliers mistake as max length for Tweets should be **280**

In [64]:
df_sub = df.loc[df['tweet_length']>280]


The longest twitter has 511 according to our previous lambda function. However, this seems to be off as the actual tweet is significantly shorter than that.

In [65]:
df_sub.sort_values(by=['tweet_length'], ascending = False)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length
29982,3,0,3,0,0,1,RT @TrxllLegend: One good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,rt @trxlllegend: one good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,rt @trxlllegend: one good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,1,rt : one good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,rt one good girl is worth a thousand bitches # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #,rt one good girl is worth a thousand bitches,511
28044,3,0,3,0,0,1,No summer school? &#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#128515;&#1

There seems to be something off with the spacing in each tweet. Removing all spaces within characters.

In [66]:
# removing spaces in tweets to count only characters
df['nospaces'] = df['no_handle_no_special_no_sin_ash'].str.replace(" ", "")

Counting the total characters without spacing in order to get reliable information of the length of actual text in tweets

In [67]:
df['character_count'] = df['nospaces'].apply(lambda x: len(x))
df.head(50)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere,0,13 days to go #gettingthere,days to go #gettingthere,days to go #gettingthere,30,daystogo#gettingthere,21
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag,1,get off my twitter fag,get off my twitter fag,get off my twitter fag,23,getoffmytwitterfag,18
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery # # #,these hoes got more bodies than a cemetery,58,thesehoesgotmorebodiesthanacemetery,35
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the racists cuz she's not white.,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,124,afriendjusttoldmeshe'safraidtogotodcrallyampbeattackedby#berniebrosortheracistscuzshe'snotwhite,95
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,1,dubya hates todays teabagger movement.,dubya hates todays teabagger movement,dubya hates todays teabagger movement,39,dubyahatestodaysteabaggermovement,33
5,3,0,0,0,3,3,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,1,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,117,i'venoticedalotof#icontfpresentationsmentionhappinesswonderifprofessionhasaboveaveragehappiness,95
6,3,0,0,0,3,3,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce,0,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows #country #music #lylepierce,getting for this weekends shows #country #music #lylepierce,62,gettingforthisweekendsshows#country#music#lylepierce,52
7,3,0,0,0,3,3,@user my final legislative session day has officially begun! @user @user #albany,@user my final legislative session day has officially begun! @user @user #albany,@user my final legislative session day has officially begun! @user @user #albany,3,my final legislative session day has officially begun! #albany,my final legislative session day has officially begun #albany,my final legislative session day has officially begun #albany,69,myfinallegislativesessiondayhasofficiallybegun#albany,53
8,3,2,1,0,0,0,@lucas

The longest tweet in the dataset according to the character length field had 511 characters. We can see from the third row in the following df that the issue has been solved as the new character count field is correctly showing 36 for the total amount of characters.

In [68]:
# double checking issue is fixed with the longest tweet from previous length
checking = df[df['tweet_low'].str.contains(r'one good girl')]
checking.head(5)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count
9691,3,0,3,0,0,1,"Close your eyes and let the world paint a thousand pictures, one good girl is worth a thousand bitches...","close your eyes and let the world paint a thousand pictures, one good girl is worth a thousand bitches...","close your eyes and let the world paint a thousand pictures, one good girl is worth a thousand bitches...",0,"close your eyes and let the world paint a thousand pictures, one good girl is worth a thousand bitches...",close your eyes and let the world paint a thousand pictures one good girl is worth a thousand bitches,close your eyes and let the world paint a thousand pictures one good girl is worth a thousand bitches,105,closeyoureyesandlettheworldpaintathousandpicturesonegoodgirlisworthathousandbitches,83
21028,3,0,3,0,0,1,one good girl is worth 1000 bitches,one good girl is worth 1000 bitches,one good girl is worth 1000 bitches,0,one good girl is worth 1000 bitches,one good girl is worth bitches,one good girl is worth bitches,35,onegoodgirlisworthbitches,25
29982,3,0,3,0,0,1,RT @TrxllLegend: One good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,rt @trxlllegend: one good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,rt @trxlllegend: one good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#8230;,1,rt : one good girl is worth a thousand bitches\n\n&#128112; = &#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#128109;&#12

As we can see from above all length are now consistent and in line with the actual number of charcater text in a tweet.
Also, there seems to be no missing values at this point.

In [69]:
df.isna().sum()

count                              0
hate_speech                        0
offensive_language                 0
neither                            0
positive                           0
class                              0
tweet                              0
tweet_low                          0
no_url                             0
handle_count                       0
no_handle                          0
no_handle_no_special               0
no_handle_no_special_no_sin_ash    0
tweet_length                       0
nospaces                           0
character_count                    0
dtype: int64

##  Lemmatization with POS

In this section, we are lemmatizing words with part-of-speech (POS). 

By doing this, each word is labelled as either a noun, verb, adjective or adverb.

Each sentence is first tokenized and associated with a tag based off the lexical database Wordnet. Tuples of tokens and wordnet tags are then created and if there's a match the word is lemmatized according to the tag content(noun,verb etc) otherwise it's left as is.

There were few exceptions that I had to manually tweak. For instance the word 'ass' was for some reason lemmatized to 'as'. The full code is in the functions folder.


In [71]:
df['lemmatized'] = df['no_handle_no_special_no_sin_ash'].apply(lambda x: lemmatize_sentence(x))

In [72]:
df.head(10)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere,0,13 days to go #gettingthere,days to go #gettingthere,days to go #gettingthere,30,daystogo#gettingthere,21,day to go # gettingthere
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag,1,get off my twitter fag,get off my twitter fag,get off my twitter fag,23,getoffmytwitterfag,18,get off my twitter fag
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery # # #,these hoes got more bodies than a cemetery,58,thesehoesgotmorebodiesthanacemetery,35,these hoe get more body than a cemetery
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the racists cuz she's not white.,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,124,afriendjusttoldmeshe'safraidtogotodcrallyampbeattackedby#berniebrosortheracistscuzshe'snotwhite,95,a friend just tell me she 's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she 's not white
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,1,dubya hates todays teabagger movement.,dubya hates todays teabagger movement,dubya hates todays teabagger movement,39,dubyahatestodaysteabaggermovement,33,dubya hat today teabagger movement
5,3,0,0,0,3,3,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,1,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,117,i'venoticedalotof#icontfpresentationsmentionhappinesswonderifprofessionhasaboveaveragehappiness,95,i 've notice a lot of # icontf presentation mention happiness wonder if profession have above average happiness
6,3,0,0,0,3,3,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows! #country #music #lylepierce,0,getting for this weekends shows! #country #music #lylepierce,getting for this weekends shows #country #music #lylepierce,getting for this weekends shows #country #music #lylepierce,62,gettingforthisweekendsshows#country#music#lylepierce,52,get for this weekend show # country # music # lylepierce
7,3,0,0,0,3,3,@user my final legislative session day has officially begun! @user @user #albany,@user my 

##  Bridging all spaces between ashtags and apostrophes

In [73]:
# removing spaces in between ashtags and words
df['lemmatized_1'] = df['lemmatized'].str.replace('# ', '#')
df.head(5)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized,lemmatized_1
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere,0,13 days to go #gettingthere,days to go #gettingthere,days to go #gettingthere,30,daystogo#gettingthere,21,day to go # gettingthere,day to go #gettingthere
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag,1,get off my twitter fag,get off my twitter fag,get off my twitter fag,23,getoffmytwitterfag,18,get off my twitter fag,get off my twitter fag
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery # # #,these hoes got more bodies than a cemetery,58,thesehoesgotmorebodiesthanacemetery,35,these hoe get more body than a cemetery,these hoe get more body than a cemetery
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the racists cuz she's not white.,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,124,afriendjusttoldmeshe'safraidtogotodcrallyampbeattackedby#berniebrosortheracistscuzshe'snotwhite,95,a friend just tell me she 's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she 's not white,a friend just tell me she 's afraid to go to dc rally amp be attack by #berniebros or the racist cuz she 's not white
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,1,dubya hates todays teabagger movement.,dubya hates todays teabagger movement,dubya hates todays teabagger movement,39,dubyahatestodaysteabaggermovement,33,dubya hat today teabagger movement,dubya hat today teabagger movement


In [74]:
df['lemmatized_final'] = df['lemmatized'].str.replace(" '" ,"'")
df.head(15)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized,lemmatized_1,lemmatized_final
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere,0,13 days to go #gettingthere,days to go #gettingthere,days to go #gettingthere,30,daystogo#gettingthere,21,day to go # gettingthere,day to go #gettingthere,day to go # gettingthere
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag,1,get off my twitter fag,get off my twitter fag,get off my twitter fag,23,getoffmytwitterfag,18,get off my twitter fag,get off my twitter fag,get off my twitter fag
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery # # #,these hoes got more bodies than a cemetery,58,thesehoesgotmorebodiesthanacemetery,35,these hoe get more body than a cemetery,these hoe get more body than a cemetery,these hoe get more body than a cemetery
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the racists cuz she's not white.,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,124,afriendjusttoldmeshe'safraidtogotodcrallyampbeattackedby#berniebrosortheracistscuzshe'snotwhite,95,a friend just tell me she 's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she 's not white,a friend just tell me she 's afraid to go to dc rally amp be attack by #berniebros or the racist cuz she 's not white,a friend just tell me she's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she's not white
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,1,dubya hates todays teabagger movement.,dubya hates todays teabagger movement,dubya hates todays teabagger movement,39,dubyahatestodaysteabaggermovement,33,dubya hat today teabagger movement,dubya hat today teabagger movement,dubya hat today teabagger movement
5,3,0,0,0,3,3,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,1,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,117,i'venoticedalotof#icontfpresentationsmentionhappinesswonderifprofessionhasaboveaveragehappiness,95,i 've notice a lot of # icontf presentation mention happiness wonder if profession have above average happiness,i 've notice a lot of #icontf presentation mention happiness wonder if profession have abo

##  Removing StopWords and also words shorter than two characters

At this stage we are removing all the stopwords in our text plus words shorter than two characters as they are usually not very helpful at giving meaningful information about the text. An exception to this are negations which will be handled in a different way further down the line.

In [75]:
# importing english stopwords
stop = stopwords.words('english')
df['tweet_without_stopwords'] = df['lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [76]:
# checking whether stopwords are still there
df.head(10)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized,lemmatized_1,lemmatized_final,tweet_without_stopwords
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere,0,13 days to go #gettingthere,days to go #gettingthere,days to go #gettingthere,30,daystogo#gettingthere,21,day to go # gettingthere,day to go #gettingthere,day to go # gettingthere,day go # gettingthere
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag,1,get off my twitter fag,get off my twitter fag,get off my twitter fag,23,getoffmytwitterfag,18,get off my twitter fag,get off my twitter fag,get off my twitter fag,get twitter fag
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery # # #,these hoes got more bodies than a cemetery,58,thesehoesgotmorebodiesthanacemetery,35,these hoe get more body than a cemetery,these hoe get more body than a cemetery,these hoe get more body than a cemetery,hoe get body cemetery
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the racists cuz she's not white.,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,124,afriendjusttoldmeshe'safraidtogotodcrallyampbeattackedby#berniebrosortheracistscuzshe'snotwhite,95,a friend just tell me she 's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she 's not white,a friend just tell me she 's afraid to go to dc rally amp be attack by #berniebros or the racist cuz she 's not white,a friend just tell me she's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she's not white,friend tell 's afraid go dc rally amp attack # berniebros racist cuz 's white
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,1,dubya hates todays teabagger movement.,dubya hates todays teabagger movement,dubya hates todays teabagger movement,39,dubyahatestodaysteabaggermovement,33,dubya hat today teabagger movement,dubya hat today teabagger movement,dubya hat today teabagger movement,dubya hat today teabagger movement
5,3,0,0,0,3,3,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,1,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness?,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,117,i'venoticedalotof#icontfpresentationsmentionhappinesswonderifprofessionhasaboveaveragehappiness,95,i 've

In [77]:
# removing words shorter than two characters as they probably do not have much meaning
df['tweet_without_stopwords_and_2'] = df['tweet_without_stopwords'].apply(lambda x: ' '.join([word for word in x.split() if len(word)>2])) 

In [78]:
# sanity check
df.head(5)

,count,hate_speech,offensive_language,neither,positive,class,tweet,tweet_low,no_url,handle_count,no_handle,no_handle_no_special,no_handle_no_special_no_sin_ash,tweet_length,nospaces,character_count,lemmatized,lemmatized_1,lemmatized_final,tweet_without_stopwords,tweet_without_stopwords_and_2
0,3,0,0,0,3,3,13 days to go #gettingthere,13 days to go #gettingthere,13 days to go #gettingthere,0,13 days to go #gettingthere,days to go #gettingthere,days to go #gettingthere,30,daystogo#gettingthere,21,day to go # gettingthere,day to go #gettingthere,day to go # gettingthere,day go # gettingthere,day gettingthere
1,3,1,2,0,0,1,@anggxo get off my twitter fag,@anggxo get off my twitter fag,@anggxo get off my twitter fag,1,get off my twitter fag,get off my twitter fag,get off my twitter fag,23,getoffmytwitterfag,18,get off my twitter fag,get off my twitter fag,get off my twitter fag,get twitter fag,get twitter fag
2,3,1,2,0,0,1,These hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,these hoes got more bodies than a cemetery # # #,these hoes got more bodies than a cemetery,58,thesehoesgotmorebodiesthanacemetery,35,these hoe get more body than a cemetery,these hoe get more body than a cemetery,these hoe get more body than a cemetery,hoe get body cemetery,hoe get body cemetery
3,3,0,0,0,3,3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the racists cuz she's not white.,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,124,afriendjusttoldmeshe'safraidtogotodcrallyampbeattackedby#berniebrosortheracistscuzshe'snotwhite,95,a friend just tell me she 's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she 's not white,a friend just tell me she 's afraid to go to dc rally amp be attack by #berniebros or the racist cuz she 's not white,a friend just tell me she's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she's not white,friend tell 's afraid go dc rally amp attack # berniebros racist cuz 's white,friend tell afraid rally amp attack berniebros racist cuz white
4,3,0,0,3,0,2,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,@ramaxe1965 dubya hates todays teabagger movement.,1,dubya hates todays teabagger movement.,dubya hates todays teabagger movement,dubya hates todays teabagger movement,39,dubyahatestodaysteabaggermovement,33,dubya hat today teabagger movement,dubya hat today teabagger movement,dubya hat today teabagger movement,dubya hat today teabagger movement,dubya hat today teabagger movement


#  Labels for Binary Classification


In this final section all Tweets will be re-labelled as, for now, this project is only trying to identify negative tweets from positive/neutral ones.
We don't need the different shades between offensive and hate speech for the time being. All tweets will be re-labelled as 1 for negative and 0 for positive.

In [79]:
df['positive'].value_counts()

0    24773
3    21421
Name: positive, dtype: int64

In [80]:
# excluding neutral tweets because we are building a binary model for now
df = df.loc[df['class'] !=2 ]

In [81]:
# check length of df. It should 46194 - neutrals (4163) = 42031
len(df)

42031

In [82]:
df['neg_label'] = df['class'].apply(lambda x: 0 if x == 3 else 1)

In [83]:
df['neg_label'].value_counts()

0    21421
1    20610
Name: neg_label, dtype: int64

# Missing Values, Dropping Columns and Save for EDA

Removing all the excess columns that showed the pruning process as intermediate steps. They are not really necessary for the next step with exploratory data analysis.

In [84]:
df.drop(['tweet','count','hate_speech','offensive_language','neither',
         'positive','class', 'tweet','no_url', 'no_handle','no_handle_no_special',
         'nospaces',], axis = 1, inplace=True)

In [85]:
df.head(5)

,tweet_low,handle_count,no_handle_no_special_no_sin_ash,tweet_length,character_count,lemmatized,lemmatized_1,lemmatized_final,tweet_without_stopwords,tweet_without_stopwords_and_2,neg_label
0,13 days to go #gettingthere,0,days to go #gettingthere,30,21,day to go # gettingthere,day to go #gettingthere,day to go # gettingthere,day go # gettingthere,day gettingthere,0
1,@anggxo get off my twitter fag,1,get off my twitter fag,23,18,get off my twitter fag,get off my twitter fag,get off my twitter fag,get twitter fag,get twitter fag,1
2,these hoes got more bodies than a cemetery&#128056;&#9749;&#65039;.,0,these hoes got more bodies than a cemetery,58,35,these hoe get more body than a cemetery,these hoe get more body than a cemetery,these hoe get more body than a cemetery,hoe get body cemetery,hoe get body cemetery,1
3,a friend just told me she's afraid to go to dc rally &amp; be attacked by #berniebros or the @user racists cuz she's not white.,1,a friend just told me she's afraid to go to dc rally amp be attacked by #berniebros or the racists cuz she's not white,124,95,a friend just tell me she 's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she 's not white,a friend just tell me she 's afraid to go to dc rally amp be attack by #berniebros or the racist cuz she 's not white,a friend just tell me she's afraid to go to dc rally amp be attack by # berniebros or the racist cuz she's not white,friend tell 's afraid go dc rally amp attack # berniebros racist cuz 's white,friend tell afraid rally amp attack berniebros racist cuz white,0
5,i've noticed a lot of #icontf16 presentations mention happiness. wonder if profession has above average happiness? @user,1,i've noticed a lot of #icontf presentations mention happiness wonder if profession has above average happiness,117,95,i 've notice a lot of # icontf presentation mention happiness wonder if profession have above average happiness,i 've notice a lot of #icontf presentation mention happiness wonder if profession have above average happiness,i've notice a lot of # icontf presentation mention happiness wonder if profession have above average happiness,'ve notice lot # icontf presentation mention happiness wonder profession average happiness,'ve notice lot icontf presentation mention happiness wonder profession average happiness,0


In [86]:
df.isna().sum()

tweet_low                          0
handle_count                       0
no_handle_no_special_no_sin_ash    0
tweet_length                       0
character_count                    0
lemmatized                         0
lemmatized_1                       0
lemmatized_final                   0
tweet_without_stopwords            0
tweet_without_stopwords_and_2      0
neg_label                          0
dtype: int64

In [87]:
# final length of the dataset for EDA
len(df)

42031

In [88]:
# saving file
df.to_csv(r'../data/processed/cleaned_data_2.csv', index = False)